In [1]:
import xml.etree.ElementTree as ET
from os import listdir
import numpy as np
import pandas as pd
import re
from tqdm import tqdm
from difflib import SequenceMatcher

# Split text and tag

Smoker and Family_hist can have no tag

In [2]:
file_path = "E:/JHU/课程/datadesign/NLP/data/training-RiskFactors-Complete-Set1/"
names = [f for f in listdir(file_path) if f.endswith('.xml')]
train = pd.DataFrame(np.zeros((len(names), 2), dtype=object), columns=['text', 'annotation'])
print(train.shape)
n = 0
for name in names:
    tree = ET.parse(file_path + name)
    root = tree.getroot()

    ## Get the text
    nt = re.sub('\n',' ',root[0].text)
    nt = re.sub('\t',' ',nt) 
    nt = re.sub('"',"'",nt)
    ## sample 214 has a weird character
    nt = re.sub('>','&gt;',nt) 
    nt = re.sub('<','&lt;',nt)
    train['text'][n] = nt
    n+=1


(521, 2)


In [3]:
n = 0
for name in names:
    tree = ET.parse(file_path + name)
    root = tree.getroot()
    ## Get the labels
    # skip FAMILY_HIST: doesn't have a text
    #skip = [root[1][x].tag for x in range(len(root[1]))].index('FAMILY_HIST')
    PHI = [root[1][x].tag for x in range(len(root[1]))].index('PHI') 
    # get all labels except for PHI
    tag_list = []
    for k in range(len(root[1])):
        for m in range(len(root[1][k])):
            if root[1][k][m].attrib.keys().__contains__('text') == False:
                continue
            tag_list.append((root[1][k][m].attrib['text'],root[1][k][m].tag))
    # get PHI labels
    for k in range(PHI,len(root[1])):
        tag_list.append((root[1][k].attrib['text'],root[1][k].tag))
    train['annotation'][n] = tag_list
    n+=1

train.head()
    




,text,annotation
0,Record date: 2067-05-03 Narrative History ...,"[(ZESTRIL, MEDICATION), (ZESTRIL , MEDICATION)..."
1,Record date: 2068-12-05 Narrative History ...,"[(Zestril, MEDICATION), (Zestril (LISINOPRIL),..."
2,Record Date: 2070-12-01 Narrative History ...,"[(Zestril, MEDICATION), (Zestril , MEDICATION)..."
3,Record date: 2072-07-27 Narrative History...,"[(Zestril (LISINOPRIL), MEDICATION), (Zestril,..."
4,Record date: 2075-01-31 Narrative History...,"[(Zestril (LISINOPRIL), MEDICATION), (Zestril,..."


# Functions to convert dataframe into input format
## use wisely, very slow (520 files takes 40 min to create labels)
 Please use create_labs function instead of mark_sentences


In [4]:
def matcher(string, pattern):
    '''
    Return the start and end index of any pattern present in the text.
    '''
    match_list = []
    pattern = pattern.strip()
    seqMatch = SequenceMatcher(None, string, pattern, autojunk=False)
    match = seqMatch.find_longest_match(0, len(string), 0, len(pattern))
    if (match.size == len(pattern)):
        start = match.a
        end = match.a + match.size
        match_tup = (start, end)
        string = string.replace(pattern, "X" * len(pattern), 1)
        match_list.append(match_tup)
        
    return match_list, string


def mark_sentence(s, match_list):
    '''
    Marks all the entities in the sentence as per the BIO scheme. 
    '''
    word_dict = {}
    for word in s.split():
        word_dict[word] = 'O'
        
    for start, end, e_type in match_list:
        temp_str = s[start:end]
        tmp_list = temp_str.split()
        if len(tmp_list) > 1:
            word_dict[tmp_list[0]] = 'B-' + e_type
            for w in tmp_list[1:]:
                word_dict[w] = 'I-' + e_type
        else:
            word_dict[temp_str] = 'B-' + e_type
    return word_dict

## replace !!mark_sentence!! to better label the text with more than one word
def create_labs(s,match_list):
    labs = ['O' for i in range(len(s.split()))]
    word_dict = pd.DataFrame({'word':s.split(),'label':labs})

    for start, end, e_type in match_list:
        index = len(re.findall(r' +',s[0:start]))-1
        num_words = len(s[start:end].split())

        if num_words > 1:
            word_dict.loc[index,'label'] = 'B-' + e_type
            for i in range(1,num_words):
                word_dict.loc[index+i,'label'] = 'I-' + e_type
        else:
            word_dict.loc[index,'label'] = 'B-' + e_type
    return word_dict

def clean(text):
    '''
    Just a helper fuction to add a space before the punctuations for better tokenization
    '''
    filters = ["!", "#", "$", "%", "&", "(", ")", "/", "*", ".", ":", ";", "<", "=", ">", "?", "@", "[",
               "\\", "]", "_", "`", "{", "}", "~", "'"]
    for i in text:
        if i in filters:
            text = text.replace(i, " " + i)
            
    return text

def create_data(df, filepath):
    '''
    The function responsible for the creation of data in the said format.
    '''

    with open(filepath , 'w') as f:
        for text, annotation in zip(df.text, df.annotation): 
            #text = clean(text)
            #text_ = text    
            print(train.index[train['text']== text].tolist())    
            match_list = []
            for i in annotation: 
                a, text_ = matcher(text, i[0])
                match_list.append((a[0][0], a[0][1], i[1]))
        
            d = create_labs(text, match_list)

            #for i in d.keys():
            #    f.writelines(i + ' ' + d[i] +'\n')
            #f.writelines('\n')
            for i in range(d.shape[0]):
                f.writelines(d['word'][i] + ' ' + d['label'][i] +'\n')
            f.writelines('\n')
def main():
    ## An example dataframe.
    #data = pd.DataFrame([["Horses are too tall and they pretend to care about your feelings", [("Horses", "ANIMAL")]],
    #              ["Who is Shaka Khan?", [("Shaka Khan", "PERSON")]],
    #              ["I like London and Berlin.", [("London", "LOCATION"), ("Berlin", "LOCATION")]],
    #              ["There is a banyan tree in the courtyard", [("banyan tree", "TREE")]]], columns=['text', 'annotation'])
    data = train.copy()
    #print(data.head())
    ## path to save the txt file.
    filepath = 'E:/JHU/课程/datadesign/NLP/machine_learning/tttt.txt'
    ## creating the file.
    create_data(data, filepath)
    
    
if __name__ == '__main__':
    main()

[0]
[1]
[2]
[3]
[4]
[5]
[6]
[7]
[8]
[9]
[10]
[11]
[12]
[13]
[14]
[15]
[16]
[17]
[18]
[19]
[20]
[21]
[22]
[23]
[24]
[25]
[26]
[27]
[28]
[29]
[30]
[31]
[32]
[33]
[34]
[35]
[36]
[37]
[38]
[39]
[40]
[41]
[42]
[43]
[44]
[45]
[46]
[47]
[48]
[49]
[50]
[51]
[52]
[53]
[54]
[55]
[56]
[57]
[58]
[59]
[60]
[61]
[62]
[63]
[64]
[65]
[66]
[67]
[68]
[69]
[70]
[71]
[72]
[73]
[74]
[75]
[76]
[77]
[78]
[79]
[80]
[81]
[82]
[83]
[84]
[85]
[86]
[87]
[88]
[89]
[90]
[91]
[92]
[93]
[94]
[95]
[96]
[97]
[98]
[99]
[100]
[101]
[102]
[103]
[104]
[105]
[106]
[107]
[108]
[109]
[110]
[111]
[112]
[113]
[114]
[115]
[116]
[117]
[118]
[119]
[120]
[121]
[122]
[123]
[124]
[125]
[126]
[127]
[128]
[129]
[130]
[131]
[132]
[133]
[134]
[135]
[136]
[137]
[138]
[139]
[140]
[141]
[142]
[143]
[144]
[145]
[146]
[147]
[148]
[149]
[150]
[151]
[152]
[153]
[154]
[155]
[156]
[157]
[158]
[159]
[160]
[161]
[162]
[163]
[164]
[165]
[166]
[167]
[168]
[169]
[170]
[171]
[172]
[173]
[174]
[175]
[176]
[177]
[178]
[179]
[180]
[181]
[182]
[183]
[184]


In [6]:
#read txt file
def read_txt(filename):
    with open(filename, 'r') as f:
        data = f.readlines()
    return data

list = read_txt('E:/JHU/课程/datadesign/NLP/machine_learning/trainingdata.txt')
#print(list)
for i in range(len(list)):
    list[i] = list[i].split()
#print(list)
dataframe = pd.DataFrame(list,columns=['text','label'])
print(dataframe.shape)
dataframe.head()


(336882, 2)


,text,label
0,Record,O
1,date:,O
2,2067-05-03,B-PHI
3,Narrative,O
4,History,O


In [98]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import Perceptron
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
df = dataframe.fillna(method='ffill')
df.text.nunique(), df.label.nunique()

(42418, 19)

In [99]:
df.groupby('label').size().reset_index(name='counts')

,label,counts
0,B-CAD,839
1,B-DIABETES,704
2,B-FAMILY_HIST,10
3,B-HYPERLIPIDEMIA,334
4,B-HYPERTENSION,831
5,B-MEDICATION,2937
6,B-OBESE,153
7,B-PHI,8455
8,B-SMOKER,511
9,I-CAD,2903


In [140]:
X = df.drop('label',axis=1)
print (X)

v = DictVectorizer(sparse=True)
X = v.fit_transform(X.to_dict('records'))
print(X)
y = df.label.values

classes = np.unique(y)
classes = classes.tolist()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=0)
X_train.shape, y_train.shape

              text
0           Record
1            date:
2       2067-05-03
3        Narrative
4          History
...            ...
336877          MD
336878      PGY-1,
336879       Pager
336880      #65261
336881      #65261

[336882 rows x 1 columns]
  (0, 21175)	1.0
  (1, 28179)	1.0
  (2, 6127)	1.0
  (3, 18798)	1.0
  (4, 16015)	1.0
  (5, 8799)	1.0
  (6, 42331)	1.0
  (7, 42122)	1.0
  (8, 42042)	1.0
  (9, 36874)	1.0
  (10, 30743)	1.0
  (11, 30229)	1.0
  (12, 21972)	1.0
  (13, 32275)	1.0
  (14, 12678)	1.0
  (15, 37930)	1.0
  (16, 33491)	1.0
  (17, 33143)	1.0
  (18, 41914)	1.0
  (19, 25101)	1.0
  (20, 11665)	1.0
  (21, 5478)	1.0
  (22, 22888)	1.0
  (23, 26636)	1.0
  (24, 41421)	1.0
  :	:
  (336857, 25101)	1.0
  (336858, 37704)	1.0
  (336859, 33662)	1.0
  (336860, 20619)	1.0
  (336861, 19897)	1.0
  (336862, 15167)	1.0
  (336863, 35170)	1.0
  (336864, 32377)	1.0
  (336865, 13574)	1.0
  (336866, 11176)	1.0
  (336867, 11664)	1.0
  (336868, 12975)	1.0
  (336869, 21987)	1.0
  (336870, 36943

((269505, 42418), (269505,))

In [148]:
per = Perceptron(verbose=10, n_jobs=-1, max_iter=5) #this is the perceptron model
per.partial_fit(X_train, y_train,classes=classes)

-- Epoch 1
-- Epoch 1
-- Epoch 1
-- Epoch 1
-- Epoch 1
Norm: 12.81, NNZs: 164, Bias: -0.060000, T: 269505, Avg. loss: 0.001726
Total training time: 0.04 seconds.
-- Epoch 1
-- Epoch 1
-- Epoch 1
Norm: 8.94, NNZs: 80, Bias: -0.080000, T: 269505, Avg. loss: 0.000349
Total training time: 0.03 seconds.
-- Epoch 1
-- Epoch 1
-- Epoch 1
-- Epoch 1
Norm: 28.34, NNZs: 803, Bias: -0.090000, T: 269505, Avg. loss: 0.002359
Total training time: 0.04 seconds.
Norm: 16.00, NNZs: 256, Bias: -0.020000, T: 269505, Avg. loss: 0.001099
Total training time: 0.05 seconds.
Norm: 9.90, NNZs: 98, Bias: -0.040000, T: 269505, Avg. loss: 0.001062
Total training time: 0.04 seconds.
Norm: 2.00, NNZs: 4, Bias: -0.020000, T: 269505, Avg. loss: 0.000026
Total training time: 0.05 seconds.
Norm: 6.78, NNZs: 46, Bias: -0.020000, T: 269505, Avg. loss: 0.000162
Total training time: 0.07 seconds.
Norm: 88.53, NNZs: 7837, Bias: -0.030000, T: 269505, Avg. loss: 0.004867
Total training time: 0.05 seconds.
-- Epoch 1
-- Epoch 

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of  19 | elapsed:    0.0s remaining:    0.7s
[Parallel(n_jobs=-1)]: Done   4 out of  19 | elapsed:    0.0s remaining:    0.3s
[Parallel(n_jobs=-1)]: Done   6 out of  19 | elapsed:    0.0s remaining:    0.2s
[Parallel(n_jobs=-1)]: Done   8 out of  19 | elapsed:    0.0s remaining:    0.1s
[Parallel(n_jobs=-1)]: Done  10 out of  19 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  12 out of  19 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  14 out of  19 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  16 out of  19 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  19 out of  19 | elapsed:    0.1s finished


Perceptron(max_iter=5, n_jobs=-1, verbose=10)

In [149]:
new_classes = classes.copy()
new_classes.pop()
new_classes

['B-CAD',
 'B-DIABETES',
 'B-FAMILY_HIST',
 'B-HYPERLIPIDEMIA',
 'B-HYPERTENSION',
 'B-MEDICATION',
 'B-OBESE',
 'B-PHI',
 'B-SMOKER',
 'I-CAD',
 'I-DIABETES',
 'I-FAMILY_HIST',
 'I-HYPERLIPIDEMIA',
 'I-HYPERTENSION',
 'I-MEDICATION',
 'I-OBESE',
 'I-PHI',
 'I-SMOKER']

In [157]:
print (classification_report(y_pred=per.predict(X_test), y_true=y_test, labels=classes))

                  precision    recall  f1-score   support

           B-CAD       0.18      0.18      0.18       171
      B-DIABETES       0.56      0.36      0.44       141
   B-FAMILY_HIST       0.00      0.00      0.00         2
B-HYPERLIPIDEMIA       0.49      0.28      0.36        60
  B-HYPERTENSION       0.36      0.35      0.35       195
    B-MEDICATION       0.72      0.63      0.67       563
         B-OBESE       0.58      0.58      0.58        36
           B-PHI       0.62      0.29      0.39      1692
        B-SMOKER       0.52      0.17      0.25       103
           I-CAD       0.24      0.14      0.18       580
      I-DIABETES       0.03      0.37      0.05       155
   I-FAMILY_HIST       0.00      0.00      0.00        15
I-HYPERLIPIDEMIA       0.17      0.08      0.11        12
  I-HYPERTENSION       0.03      0.02      0.02       122
    I-MEDICATION       0.26      0.19      0.22       298
         I-OBESE       0.00      0.00      0.00         7
           I-

In [158]:
#support vector machine
svm = SGDClassifier(alpha=.0001, max_iter=50,penalty="elasticnet")
svm.partial_fit(X_train, y_train,classes=classes)

print (classification_report(y_pred=svm.predict(X_test), y_true=y_test, labels=classes))

c:\Users\admin\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                  precision    recall  f1-score   support

           B-CAD       0.00      0.00      0.00       171
      B-DIABETES       0.80      0.14      0.24       141
   B-FAMILY_HIST       0.00      0.00      0.00         2
B-HYPERLIPIDEMIA       0.88      0.25      0.39        60
  B-HYPERTENSION       0.79      0.10      0.17       195
    B-MEDICATION       0.79      0.14      0.24       563
         B-OBESE       0.00      0.00      0.00        36
           B-PHI       0.00      0.00      0.00      1692
        B-SMOKER       0.00      0.00      0.00       103
           I-CAD       0.00      0.00      0.00       580
      I-DIABETES       0.95      0.12      0.22       155
   I-FAMILY_HIST       0.00      0.00      0.00        15
I-HYPERLIPIDEMIA       0.00      0.00      0.00        12
  I-HYPERTENSION       0.00      0.00      0.00       122
    I-MEDICATION       0.00      0.00      0.00       298
         I-OBESE       0.00      0.00      0.00         7
           I-

c:\Users\admin\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\admin\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [159]:
#naive bayes
nb = MultinomialNB(alpha=.01)
nb.partial_fit(X_train, y_train,classes=classes)

print (classification_report(y_pred=nb.predict(X_test), y_true=y_test, labels=classes))

c:\Users\admin\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                  precision    recall  f1-score   support

           B-CAD       0.52      0.08      0.14       171
      B-DIABETES       0.63      0.28      0.38       141
   B-FAMILY_HIST       0.00      0.00      0.00         2
B-HYPERLIPIDEMIA       0.83      0.48      0.61        60
  B-HYPERTENSION       0.70      0.31      0.43       195
    B-MEDICATION       0.74      0.71      0.72       563
         B-OBESE       1.00      0.19      0.33        36
           B-PHI       0.68      0.32      0.43      1692
        B-SMOKER       0.80      0.12      0.20       103
           I-CAD       0.48      0.06      0.11       580
      I-DIABETES       0.69      0.17      0.28       155
   I-FAMILY_HIST       0.00      0.00      0.00        15
I-HYPERLIPIDEMIA       0.00      0.00      0.00        12
  I-HYPERTENSION       0.22      0.02      0.03       122
    I-MEDICATION       0.49      0.10      0.16       298
         I-OBESE       0.00      0.00      0.00         7
           I-

c:\Users\admin\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\admin\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [118]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline

#X_train, X_test, y_train, y_test = train_test_split(df['text'], df['label'], test_size=0.33, random_state=42)

text_clf = Pipeline([('vect', CountVectorizer()),
                        ('tfidf', TfidfTransformer()),
                        ('clf', SGDClassifier(loss='hinge', penalty='l2',
                                            alpha=1e-3, random_state=42,
                                            max_iter=5, tol=None)),
                        ])

predicted = text_clf.predict(X_test)
np.mean(predicted == y_test)

NotFittedError: Vocabulary not fitted or provided

In [117]:
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import classification_report
text_clf_svm = Pipeline([('vect', CountVectorizer()),
                        ('tfidf', TfidfTransformer()),
                        ('clf-svm', SGDClassifier(loss='hinge', penalty='l2',
                                                    alpha=1e-3, random_state=42,
                                                    max_iter=5, tol=None)),
                        ])
text_clf_svm.fit(X_train, y_train)

predicted_svm = text_clf_svm.predict(X_test)
y = classification_report(y_test, predicted_svm)
print(y)

AttributeError: lower not found

In [147]:
from sklearn.model_selection import GridSearchCV
parameters_svm = {'vect__ngram_range': [(1, 1), (1, 2)],
                    'tfidf__use_idf': (True, False),
                    'clf-svm__alpha': (1e-2, 1e-3),
                    }

gs_clf_svm = GridSearchCV(text_clf_svm, parameters_svm, n_jobs=-1)
gs_clf_svm = gs_clf_svm.fit(X_train, y_train)

gs_clf_svm.best_score_
gs_clf_svm.best_params_

ValueError: 
All the 40 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
40 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\admin\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\admin\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\pipeline.py", line 378, in fit
    Xt = self._fit(X, y, **fit_params_steps)
  File "c:\Users\admin\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\pipeline.py", line 336, in _fit
    X, fitted_transformer = fit_transform_one_cached(
  File "c:\Users\admin\AppData\Local\Programs\Python\Python310\lib\site-packages\joblib\memory.py", line 34

In [12]:
#Traing model with CRF
from sklearn_crfsuite import CRF
from sklearn_crfsuite.metrics import flat_classification_report
from sklearn.model_selection import train_test_split
from sklearn_crfsuite import metrics
from sklearn_crfsuite import scorers
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV

X = dataframe['text'].values.tolist()
y = dataframe['label'].values.tolist()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

def word2features(sent, i):
    word = sent[i]
    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
    }
    if i > 0:
        word1 = sent[i-1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
        })
    else:
        features['BOS'] = True

    if i < len(sent)-1:
        word1 = sent[i+1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
        })
    else:
        features['EOS'] = True

    return features

def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]
def sent2labels(sent):
    return [label for token, label in sent]
def sent2tokens(sent):
    return [token for token, label in sent]

X_train = [sent2features(s) for s in X_train]
X_test = [sent2features(s) for s in X_test]

crf = CRF(algorithm='lbfgs',
            c1=0.1,     
            c2=0.1,
            max_iterations=100,
            all_possible_transitions=True)

crf.fit(X_train, y_train)

y_pred = crf.predict(X_test)
print(metrics.flat_classification_report(y_test, y_pred))


TypeError: object of type 'float' has no len()

# Bunch of code for testing errors
## trash

In [ ]:
def matcher(string, pattern):
    '''
    Return the start and end index of any pattern present in the text.
    '''
    match_list = []
    pattern = pattern.strip()
    seqMatch = SequenceMatcher(None, string, pattern, autojunk=False)
    match = seqMatch.find_longest_match(0, len(string), 0, len(pattern))
    if (match.size == len(pattern)):
        start = match.a
        end = match.a + match.size
        match_tup = (start, end)
        string = string.replace(pattern, "X" * len(pattern), 1)
        match_list.append(match_tup)
        
    return match_list, string
text = train['text'][0]
annotation = train['annotation'][0]
#pattern = pattern.strip()
match_list = []
for i in annotation:
    a, text_ = matcher(text, i[0])
    match_list.append((a[0][0], a[0][1], i[1]))


match_list

In [ ]:
def create_labs(s,match_list):
    labs = ['O' for i in range(len(s.split()))]
    word_dict = pd.DataFrame({'word':s.split(),'label':labs})

    for start, end, e_type in match_list:
        index = len(re.findall(r' +',s[0:start]))-1
        num_words = len(s[start:end].split())
        index,num_words

        if num_words > 1:
            word_dict.loc[index,'label'] = 'B-' + e_type
            for i in range(1,num_words):
                word_dict.loc[index+i,'label'] = 'I-' + e_type
        else:
            word_dict.loc[index,'label'] = 'B-' + e_type
    return word_dict

x = create_labs(train['text'][0],match_list)

In [ ]:
x

## test how to find the word using index

In [ ]:
        
a = s.split()
a.index('smoking')
len(re.findall(r' +',s[0:start]))
a[86],len(s[start:end].split())
#temp_str = pd.DataFrame(s.split(), columns=['word'])
#temp_str['len'] = temp_str['word'].apply(lambda x: len(x))
#temp_str['cumsum'] = temp_str['len'].cumsum()
#temp_str

## find which label went crazy and can't match text

In [17]:
match_list = []
text = train['text'][176]
#text = "Horses are too tall and they pretend to care about your feelings"
annotation = train['annotation'][176]
for i in annotation:
    print(i)
    a, text_ = matcher(text, i[0])
    print(a)
    match_list.append((a[0][0], a[0][1], i[1]))
    


TypeError: 'int' object is not iterable

In [ ]:
train['text'][214]
#tree = ET.parse(file_path + names[62])
#root = tree.getroot()
#root[0].text

In [ ]:
names[214]

In [ ]:
tree = ET.parse(file_path + names[0])
root = tree.getroot()
## Get the labels
# skip FAMILY_HIST: doesn't have a text
#skip = [root[1][x].tag for x in range(len(root[1]))].index('FAMILY_HIST')
PHI = [root[1][x].tag for x in range(len(root[1]))].index('PHI')
# get all labels except for PHI
tag_list = []
root[1][5][0].attrib['text']

# create a list of tag as tuple

In [ ]:
from flair.data import Corpus
from flair.datasets import ColumnCorpus
# define columns
columns = {0 : 'text', 1 : 'ner'}
# directory where the data resides
data_folder = 'C:/Users/Leste/Desktop/BDD data'
# initializing the corpus
corpus: Corpus = ColumnCorpus(data_folder, columns,
                              train_file = 'train.txt')

In [ ]:
corpus.train[0].to_tagged_string('ner')

In [ ]:
file_path = "C:/Users/Leste/Desktop/BDD data/extracted/training-RiskFactors-Complete-Set1/"
names = [f for f in listdir(file_path) if f.endswith('.xml')]
tree = ET.parse(file_path + names[0])
root = tree.getroot()

## skip FAMILY_HIST: doesn't have a text
skip = [root[1][x].tag for x in range(len(root[1]))].index('FAMILY_HIST')
PHI = [root[1][x].tag for x in range(len(root[1]))].index('PHI')

## get all labels except for PHI
tag_list = []
for n in range(len(root[1])):
    if n == skip:
        continue
    for m in range(len(root[1][n])):
        tag_list.append((root[1][n][m].attrib['text'],root[1][n][m].tag))

## get PHI labels
for n in range(PHI,len(root[1])):
    tag_list.append((root[1][n].attrib['text'],root[1][n].tag))

## clean the text

In [ ]:
import re

file_path = "C:/Users/Leste/Desktop/BDD data/extracted/training-RiskFactors-Complete-Set1/"
names = [f for f in listdir(file_path) if f.endswith('.xml')]
tree = ET.parse(file_path + names[0])
root = tree.getroot()
nt = re.sub('\n','',root[0].text)



## Use trained model from flair

In [ ]:
from flair.models import SequenceTagger
from flair.tokenization import SegtokSentenceSplitter
splitter = SegtokSentenceSplitter()
tagger = SequenceTagger.load('ner')
test = text_list[names[0]]
sentences = splitter.split(test)
sentences


## Try training NER with BERT

In [ ]:
from flair.embeddings import TransformerWordEmbeddings
from flair.models import SequenceTagger
from flair.trainers import ModelTrainer
from flair.datasets import UD_ENGLISH
# 1. get the corpus
corpus = UD_ENGLISH().downsample(0.1)


In [ ]:
import pandas as pd
print(pd.DataFrame(corpus.test[0]))

In [ ]:

# 2. what label do we want to predict?
label_type = 'upos'


# 3. make the label dictionary from the corpus
label_dict = corpus.make_label_dictionary(label_type=label_type)
print(label_dict)


In [ ]:

# 4. initialize fine-tuneable transformer embeddings WITH document context
embeddings = TransformerWordEmbeddings(model='xlm-roberta-large',
                                       layers="-1",
                                       subtoken_pooling="first",
                                       fine_tune=True,
                                       use_context=True,
                                       )


In [ ]:

# 5. initialize bare-bones sequence tagger (no CRF, no RNN, no reprojection)
tagger = SequenceTagger(hidden_size=256,
                        embeddings=embeddings,
                        tag_dictionary=label_dict,
                        tag_type=label_type,
                        use_crf=False,
                        use_rnn=False,
                        reproject_embeddings=False,
                        )


In [ ]:

# 6. initialize trainer
trainer = ModelTrainer(tagger, corpus)


In [ ]:

# 7. run fine-tuning
trainer.fine_tune('resources/taggers/example-roberta',
                  learning_rate=0.1,
                  mini_batch_size=32,
                  mini_batch_chunk_size=1,  # remove this parameter to speed up computation if you have a big GPU
                  max_epochs = 10)